In [53]:
# Import dependencies
import subprocess
import json
import os
from dotenv import load_dotenv

In [63]:
# Load and set environment variables
load_dotenv()
mnemonic=os.getenv("mnemonic")
print(mnemonic)

dumb position robust coral flat tunnel mind gather must struggle motor grow all appear true


In [64]:
# Import constants.py and necessary functions from bit and web3
from constants import *
from web3 import Web3
w3= Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
from bit import Key


In [ ]:
w3.eth.getBalance("0x9018B241Bcd5Da0C65c7343cff92a738E6C647d4")

In [98]:
# Create a function called `derive_wallets`
def derive_wallets(phrase, coin, numderive):
    command = './ derive -g --mnemonic= "{mnemonic}" --cols=path,address,privkey,pubkey --numderive=numerdrive --coin={coin} --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    keys =json.loads(output)
    print(keys)

In [ ]:
coins = {'eth':derive_wallets(phrase=mnemonic,coin=ETH,numderive=3),'btc-test': derive_wallets(mnemonic,coin=BTCTEST,numderive=3)}

In [36]:
def priv_key_to_account(coin, priv_key):
    global account
    if coin == 'eth':
        account=Account.privateKeyToAccount(priv_key)
    else:
        account=PrivateKeyTestnet(priv_key)
    return account

In [39]:
def create_tx(coin,account,recipient,amount):
    global tx_data
    if coin =='eth':
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": amount}
        )
        tx_data= {
            "from": account.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address),
        }
        return tx_data
    else:
        tx_data = PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])
        return tx_data

In [ ]:
def send_tx(coin, account, recipient, amount):
    if coin == ETH:
        tx = create_tx(coin,account, recipient, amount)
        signed_tx = account.sign_transaction(tx)
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        print(result.hex())
        return result.hex()
    else:
        tx_data = create_tx(coin,account,recipient,amount)
        signed = account.sign_transaction(tx_data)
        NetworkAPI.broadcast_tx_testnet(signed)
        return signed